In [1]:
import torch
import torchvision
import numpy as np
import os
#import sys
#sys.path.append('./Functions')
from Functions import MyResNet,distortionTest

In [2]:
net = MyResNet.ResNet(MyResNet.BasicBlock,[2,2,2,2],num_classes=3)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net.to(device)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer0): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm2d(64, eps=

In [3]:
root = './data/test_images'
imgs = list(os.listdir(root))
total = len(imgs)

info = {}
all_info = []
for net_ind in range(6):

    net.load_state_dict(torch.load('./Models/best_parameter{}.pkl'.format(net_ind)))
    #i = 0
    
    for crop in [5]:
        correct = 0
        fault_list = []
        for p in imgs:
            img_path = os.path.join(root,p)
            predicted = distortionTest.distortionTest(net,device,img_path,crop=crop)
            pred = predicted.cpu().data.numpy()
            result = np.argmax(np.bincount(pred))

            if result==int(p[7]): correct += 1
            else: fault_list.append((p,result))

            #i = i+1
            #if i%100 == 0: print('Process: {}/{}'.format(i,total))
        accuracy = correct/total
        info['Net_ind'] = net_ind
        info['Crop'] = crop
        info['Accuracy'] = accuracy
        info['Faults'] = fault_list
        
        all_info.append(info.copy())

        print('net{}-crop{}-Accuracy = '.format(net_ind,crop),100*correct/total,'%')

TypeError: pic should be PIL Image or ndarray. Got <class 'tuple'>

In [ ]:
import pandas as pd
import xlwt
all_info_ = pd.DataFrame(all_info)
file_path = pd.ExcelWriter('./TestInfo/nets_test_info.xlsx')
#替换空单元格
all_info_.fillna(' ',inplace = True)
#输出
all_info_.to_excel(file_path,encoding = 'utf-8',index = False)
#保存表格
file_path.save()